In [1]:
import pandas as pd
import numpy as np
import os
import shutil
import pyreadstat
import gc

print("--- STARTING FINAL DATA ASSEMBLY ---")

# --- STEP 1: SETUP ---
data_folder = 'HCES23-24'
merge_keys = ['FSU_Serial_No', 'Panel', 'Sub_sample', 'Sample_Household_No']

household_feature_files = [
    'LEVEL - 01(Section 1 and 1.1).sav',
    'LEVEL - 03.sav',
    'LEVEL - 04 (Section 4.1).sav',
    'LEVEL - 07 (Section 4.2).sav',
    'LEVEL - 11 (Section 4.3).sav'
]
person_level_file = 'LEVEL - 02 (Section 3).sav'
level15_file = 'LEVEL - 15 (Section 1.1, A2,B2 & C2).sav'

expenditure_files = [
    'LEVEL - 05 ( Sec 5 & 6).sav', 
    'LEVEL - 06 (Section 7).sav',
    'LEVEL - 08 (Section 8.1).sav',
    'LEVEL - 09 (Section 9 & 10 & 11).sav',
    'LEVEL - 10 (Section 12).sav', 
    'LEVEL - 12 (Section 13).sav'
]


--- STARTING FINAL DATA ASSEMBLY ---


In [2]:
def reduce_mem_usage(df):
    for col in df.columns:
        col_type = df[col].dtype
        if pd.api.types.is_numeric_dtype(col_type):
            c_min, c_max = df[col].min(), df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max: df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max: df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max: df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max: df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max: df[col] = df[col].astype(np.float32)
                else: df[col] = df[col].astype(np.float64)
        elif col_type == 'object':
            if df[col].nunique() / len(df) < 0.5: df[col] = df[col].astype('category')
    return df

print("\n--- STEP 1 COMPLETE: SETUP ---")


# --- STEP 2: LOAD AND MERGE CORE HOUSEHOLD DATA ---
base_df = pd.read_spss(os.path.join(data_folder, household_feature_files[0]))
base_df = reduce_mem_usage(base_df)
base_df.drop_duplicates(subset=merge_keys, inplace=True)
for key in merge_keys:
    if key in base_df.columns: base_df[key] = pd.to_numeric(base_df[key], errors='coerce').fillna(-1).astype(int).astype(str)

for file_name in household_feature_files[1:]:
    df_to_merge = pd.read_spss(os.path.join(data_folder, file_name))
    df_to_merge = reduce_mem_usage(df_to_merge)
    df_to_merge.drop_duplicates(subset=merge_keys, inplace=True)
    for key in merge_keys:
        if key in df_to_merge.columns: df_to_merge[key] = pd.to_numeric(df_to_merge[key], errors='coerce').fillna(-1).astype(int).astype(str)
    
    new_cols = df_to_merge.columns.difference(base_df.columns)
    base_df = pd.merge(base_df, df_to_merge[merge_keys + new_cols.tolist()], on=merge_keys, how='left')
    del df_to_merge; gc.collect()

# --- NEW LOGIC: PROCESS LEVEL-15 SEPARATELY AND INTELLIGENTLY ---
df_l15 = pd.read_spss(os.path.join(data_folder, level15_file))
# Add this cell in your notebook right after loading 'LEVEL - 15.sav'
print("--- Analysis of MONTHLY_CONSUMPTION_EXP from Level 15 ---")

# Show basic statistics
print(df_l15['MONTHLY_CONSUMPTION_EXP'].describe())

# Show the most common values (this will tell us if it's mostly zero)
print("\nTop 10 most common values for MONTHLY_CONSUMPTION_EXP:")
print(df_l15['MONTHLY_CONSUMPTION_EXP'].value_counts(dropna=False).head(10))
for key in merge_keys:
    if key in df_l15.columns: df_l15[key] = pd.to_numeric(df_l15[key], errors='coerce').fillna(-1).astype(int).astype(str)

# Group by household and take the MAX value (this will grab the real number and ignore NaNs)
level15_summary = df_l15.groupby(merge_keys, observed=True)[['MONTHLY_CONSUMPTION_EXP', 'HOUSEHOLD_SIZE']].max().reset_index()
base_df = pd.merge(base_df, level15_summary, on=merge_keys, how='left')
del df_l15, level15_summary; gc.collect()

print("\n--- STEP 2 COMPLETE: CORE HOUSEHOLD DATA ASSEMBLED ---")


# --- STEP 3: PROCESS AND MERGE PERSON-LEVEL DATA ---
person_df = pd.read_spss(os.path.join(data_folder, person_level_file))
person_df = reduce_mem_usage(person_df)
person_df['Age'] = pd.to_numeric(person_df['Age'], errors='coerce')
person_df['Years_of_Education'] = pd.to_numeric(person_df['Years_of_Education'], errors='coerce')
person_df['Used_Internet_Last_30_Days'] = pd.to_numeric(person_df['Used_Internet_Last_30_Days'], errors='coerce').replace(2, 0)
for key in merge_keys:
    if key in person_df.columns: person_df[key] = pd.to_numeric(person_df[key], errors='coerce').fillna(-1).astype(int).astype(str)

household_size = person_df.groupby(merge_keys, observed=True).size().rename('household_size_calculated')
HEAD_OF_HOUSEHOLD_LABEL = 'self' 
head_age = person_df[person_df['Relation_to_Head'] == HEAD_OF_HOUSEHOLD_LABEL].groupby(merge_keys, observed=True)['Age'].first().rename('head_of_household_age')
adults_df = person_df[person_df['Age'] >= 18]
avg_edu_adults = adults_df.groupby(merge_keys, observed=True)['Years_of_Education'].mean().rename('avg_education_years_adults')
num_internet_users = person_df.groupby(merge_keys, observed=True)['Used_Internet_Last_30_Days'].sum().rename('num_internet_users')

person_agg_feats = pd.concat([household_size, head_age, avg_edu_adults, num_internet_users], axis=1).reset_index()
base_df = pd.merge(base_df, person_agg_feats, on=merge_keys, how='left')
del person_df, adults_df, person_agg_feats; gc.collect()

print("\n--- STEP 3 COMPLETE: PERSON-LEVEL FEATURES ADDED ---")


# --- STEP 4: EXTRACT SPECIFIC EXPENDITURE FEATURES (UNCHANGED) ---
# ... (The full, two-phase CSV-based code for Step 4 remains the same as the last version I sent) ...
# For brevity, I am not re-pasting the entire Step 4 block here, please use the last full version I provided.
# If you need it again, let me know.
FUEL_ITEM_CODES = [332, 338, 331, 334, 335, 341, 343, 337, 333, 344, 345, 340, 336, 342]
COMM_ITEM_CODES = [488, 487, 496, 490]
def find_column_names(df):
    item_col, value_col = None, None
    for col in df.columns:
        if 'item_code' in col.lower(): item_col = col
        if 'value' in col.lower(): value_col = col
    return item_col, value_col
temp_dir = 'cleaned_chunks_csv'
if os.path.exists(temp_dir): shutil.rmtree(temp_dir)
os.makedirs(temp_dir)
chunk_counter = 0
for file_name in expenditure_files:
    file_path = os.path.join(data_folder, file_name)
    chunk_size, offset = 1_000_000, 0
    while True:
        try:
            df_chunk, meta = pyreadstat.read_sav(file_path, row_offset=offset, row_limit=chunk_size)
            if df_chunk.empty: break
            item_col_name, value_col_name = find_column_names(df_chunk)
            if not item_col_name or not value_col_name:
                offset += chunk_size
                continue
            keys_in_chunk = [key for key in merge_keys if key in df_chunk.columns]
            df_clean_chunk = df_chunk[keys_in_chunk + [item_col_name, value_col_name]].copy()
            df_clean_chunk.rename(columns={item_col_name: 'item_code', value_col_name: 'value'}, inplace=True)
            df_clean_chunk.to_csv(os.path.join(temp_dir, f'chunk_{chunk_counter}.csv'), index=False)
            chunk_counter += 1; offset += chunk_size; del df_chunk, df_clean_chunk; gc.collect()
        except Exception as e:
            break
all_fuel_chunks, all_comm_chunks = [], []
cleaned_chunk_files = [os.path.join(temp_dir, f) for f in os.listdir(temp_dir) if f.endswith('.csv')]
for file_path in cleaned_chunk_files:
    df_clean_chunk = pd.read_csv(file_path, dtype={key: str for key in merge_keys if key in pd.read_csv(file_path, nrows=0).columns})
    all_fuel_chunks.append(df_clean_chunk[df_clean_chunk['item_code'].isin(FUEL_ITEM_CODES)].groupby(merge_keys, observed=True)['value'].sum())
    all_comm_chunks.append(df_clean_chunk[df_clean_chunk['item_code'].isin(COMM_ITEM_CODES)].groupby(merge_keys, observed=True)['value'].sum())
    del df_clean_chunk; gc.collect()
total_fuel_spending = pd.concat(all_fuel_chunks).groupby(level=list(range(len(merge_keys)))).sum()
total_comm_spending = pd.concat(all_comm_chunks).groupby(level=list(range(len(merge_keys)))).sum()
shutil.rmtree(temp_dir)
fuel_df_final = total_fuel_spending.rename('fuel_expenditure').reset_index()
comm_df_final = total_comm_spending.rename('comm_expenditure').reset_index()
for key in merge_keys:
    if key in fuel_df_final.columns: fuel_df_final[key] = pd.to_numeric(fuel_df_final[key], errors='coerce').fillna(-1).astype(int).astype(str)
    if key in comm_df_final.columns: comm_df_final[key] = pd.to_numeric(comm_df_final[key], errors='coerce').fillna(-1).astype(int).astype(str)
base_df = pd.merge(base_df, fuel_df_final, on=merge_keys, how='left')
base_df = pd.merge(base_df, comm_df_final, on=merge_keys, how='left')
base_df['fuel_expenditure'] = base_df['fuel_expenditure'].fillna(0)
base_df['comm_expenditure'] = base_df['comm_expenditure'].fillna(0)
# =========================================================================
# === NEW LOGIC: FEATURE ENGINEERING FOR COMPOSITE SCORES (X1 and X2) ===
# =========================================================================

# 1. ASSET SCORE (X1): Weighted Sum of Possessions

# Define Asset Columns and Weights (Adjust these weights based on your domain knowledge!)
asset_weights = {
    'Possess_Car': 5,
    'Possess_Truck': 5,
    'Possess_WashingMachine': 3,
    'Possess_Laptop': 3,
    'Possess_Refrigerator': 2,
    'Possess_Television': 2,
    'Possess_AirCooler': 1,
    'Possess_Bicycle': 1,
    'Possess_Scooter': 2
}

# Find all actual 'Possess_' columns present in the dataframe
possess_cols = [col for col in base_df.columns if col.startswith('Possess_')]
asset_cols_to_use = base_df.columns.intersection(asset_weights.keys()).tolist()

# Ensure binary columns are numeric (1/0) and fill NaNs (missing info) with 0
for col in asset_cols_to_use:
    base_df[col] = pd.to_numeric(base_df[col], errors='coerce').fillna(0) # Assuming HCES data is 1/0, if not map 'Yes' to 1

base_df['Asset_Score_X1'] = 0
for col in asset_cols_to_use:
    weight = asset_weights.get(col, 0)
    base_df['Asset_Score_X1'] += base_df[col] * weight

print(f"Engineered Asset_Score_X1 using {len(asset_cols_to_use)} columns.")


# 2. RATION & SCHEME INDEX (X2): Simple Sum of Social Benefits

scheme_cols = [
    'Benefitted_From_PMGKY',
    'Ayushman_beneficiary',
    'LPG_subsidized_cylinders',
    'LPG_subsidy_received',
    'Medical_benefit_received',
    'Ration_Any_Item_Last_30_Days',
    # Add other binary scheme/ration columns from your data here
]
scheme_cols_to_use = base_df.columns.intersection(scheme_cols).tolist()

# Ensure scheme columns are numeric (1/0) and fill NaNs (missing info) with 0
for col in scheme_cols_to_use:
    base_df[col] = pd.to_numeric(base_df[col], errors='coerce').fillna(0) # Assuming HCES data is 1/0

base_df['Scheme_Index_X2'] = base_df[scheme_cols_to_use].sum(axis=1)

print(f"Engineered Scheme_Index_X2 using {len(scheme_cols_to_use)} columns.")

# =========================================================================
# =========================================================================

print("\n--- STEP 4 COMPLETE: EXPENDITURE FEATURES ADDED ---")


# --- STEP 5: CALCULATE TARGET VARIABLE, CLEAN, AND SAVE ---

# a. Calculate target variable
size_col = 'household_size_calculated' if 'household_size_calculated' in base_df.columns else 'HOUSEHOLD_SIZE'
base_df['MPCE'] = base_df['MONTHLY_CONSUMPTION_EXP'] / base_df[size_col]
base_df['MPCE'] = base_df['MPCE'].replace([np.inf, -np.inf], np.nan)

# b. <<< THE CRUCIAL FIX >>>
# Drop all rows where our key target variable or its source is missing.
# This ensures the model is only trained on valid data.
print(f"\nShape before dropping missing target values: {base_df.shape}")
base_df.dropna(subset=['MONTHLY_CONSUMPTION_EXP', 'MPCE'], inplace=True)
print(f"Shape after dropping missing target values: {base_df.shape}")

# c. Save the cleaned file
base_df.to_parquet('master_dataset.parquet', index=False)

print("\n--- STEP 5 COMPLETE: TARGET VARIABLE CREATED & CLEANED FILE SAVED ---")
print("\n--- DATA ASSEMBLY IS NOW 100% COMPLETE! ---")
print("\nFinal assembled data sample:")
print(base_df[['Scheme_Index_X2','Asset_Score_X1','household_size_calculated', 'head_of_household_age', 'fuel_expenditure', 'comm_expenditure', 'MPCE']].head())

print("\n--- Descriptive Statistics of Final Assembled Data ---")
print(base_df[['Scheme_Index_X2','Asset_Score_X1','household_size_calculated', 'fuel_expenditure', 'comm_expenditure', 'MONTHLY_CONSUMPTION_EXP', 'MPCE']].describe())


--- STEP 1 COMPLETE: SETUP ---
--- Analysis of MONTHLY_CONSUMPTION_EXP from Level 15 ---
count    785859.000000
mean      12894.407002
std        7653.676532
min         200.000000
25%        8000.000000
50%       11500.000000
75%       16000.000000
max      155000.000000
Name: MONTHLY_CONSUMPTION_EXP, dtype: float64

Top 10 most common values for MONTHLY_CONSUMPTION_EXP:
MONTHLY_CONSUMPTION_EXP
NaN        261953
10000.0     48419
12000.0     45905
15000.0     44313
8000.0      39636
9000.0      30202
14000.0     26693
20000.0     25283
18000.0     24241
7000.0      24128
Name: count, dtype: int64

--- STEP 2 COMPLETE: CORE HOUSEHOLD DATA ASSEMBLED ---

--- STEP 3 COMPLETE: PERSON-LEVEL FEATURES ADDED ---
Engineered Asset_Score_X1 using 9 columns.
Engineered Scheme_Index_X2 using 6 columns.

--- STEP 4 COMPLETE: EXPENDITURE FEATURES ADDED ---

Shape before dropping missing target values: (227114, 141)
Shape after dropping missing target values: (227114, 141)

--- STEP 5 COMPLETE: TARG

In [3]:
import pandas as pd

# You might need to install a library to read parquet files if you haven't already.
# Open your Anaconda Prompt and run: pip install pyarrow

# --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- ---
# 1. Load your cleaned .parquet file
file_path = 'C:/Users/Yash Kumar/Desktop/SIH_Credit_Scoring/master_dataset.parquet' 
df = pd.read_parquet(file_path)

# 2. Get the list of all column names
column_list = df.columns.tolist()

# 3. Print the full list
print(column_list)
# --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- --- ---

['Survey_Name', 'Year', 'FSU_Serial_No', 'Sector', 'State', 'NSS_Region', 'District', 'Stratum', 'Sub_stratum', 'Panel', 'Sub_sample', 'FOD_Sub_Region', 'Sample_SU_No', 'Sample_Sub_Division_No', 'Second_Stage_Stratum_No', 'Sample_Household_No', 'Questionnaire_No', 'Level', 'Survey_Code', 'Reason_for_Substitution_Code', 'Multiplier', 'Benefitted_From_PMGKY', 'Casual_Labour_Source_Sector', 'Dwelling_Unit_Exists', 'Energy_Source_Cooking', 'Energy_Source_Lighting', 'Engaged_in_Economic_Activity_Las', 'HH_Size_FDQ', 'Household_Type', 'Land_Ownership', 'Max_Income_Activity', 'NCO_2015_Code', 'NIC_2008_Code', 'Ration_Card_Type', 'Regular_Wage_Source_Sector', 'Religion_of_HH_Head', 'Rent_Rate_Available_Rural', 'Self_Employment_Source_Sector', 'Social_Group_of_HH_Head', 'Total_Area_Land_Owned_Acres', 'Type_of_Dwelling_Unit', 'Type_of_Land_Owned', 'Ceremony_Performed_Last_30_Days', 'Meals_Served_to_Non_HH_Members', 'Online_Dry_Fruits', 'Online_Egg_Fish_Meat', 'Online_Fresh_Fruits', 'Online_Groce